# 데이터 크롤링

웹 페이지는 기본 적으로 HTML 문서로 구성되어 있다.

이외에 javascript, css도 사용되나, 데이터 크롤링에서는 데이터의 내용을 다루는 것으르로 여기서는 HTML 문서를 기본으로 살펴본다.

우선 아래와 같은 라이브러리가 필요하다

In [1]:
from bs4 import BeautifulSoup # 파이썬으로 HTML을 다루기 위한 라이브러리
import requests # 요청을 위한 라이브러리
import time # 시간을 조절하기 위한 라이브러리
import pandas as pd

크롤링할 웹페이지의 주소를 복사하여 변수에 저장한다.

(주의) 항상 메인 화면에서 무언가를 검색한 후 해당 페이지의 URL을 가져오기

네이버의 회차별 로또 번호를 제공하는 기능을 통해 로또 정보의 데이터 크롤링을 진행해보자.

로또 페이지를 가져온 URL:
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A1%9C%EB%98%90

In [ ]:
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A1%9C%EB%98%90'
res = requests.get(url)
res

<Response [200]>

웹페이지를 requests를 통해 가져왔다. 'Respense [200]'이 출력되면 정상적으로 작동한 것

In [ ]:
res.text[0:3000] # 앞에서 3000자만 출력하였다.

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="로또 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'로또\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'로또\'의 네이버 통합검색 결과입니다."> <title>로또 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/s

위와 같이 text를 불려오면 웹 페이지의 html을 가져온다. 하지만 출력결과를 잘 보면 진짜 html 문서가 아닌 양쪽에 '가 있는 text 문서이다. 이를 진짜 html 문서로 변환해보자

In [ ]:
html = BeautifulSoup(res.text)
# html # html을 출력해 보면 문자열이 아닌 html 형식대로 출력된다

## 원하는 정보 찾기

### 로또 정보 예제

#### 로또 회차 정보 찾기

웹 페이지에서 로또 회차가 나와았는 부분에서 우클릭 후 '검사'를 클릭한다.
페이지 소스에서 로또 회차 정보가 a tag 안에 들어있는 것을 확인한다.

``` html
<a onclick="return goOtherCR(this,&quot;a=nco_x5e*2&amp;r=1&amp;i=&quot;+urlencode(&quot;0011AD9E_000001E7C267&quot;)+&quot;&amp;u=&quot;+urlencode(this.href));" class="text _select_trigger _text" href="" aria-expanded="false">1078회차 (2023.07.29.)</a>
```

a 라는 tag 만으로 원하는 위치를 찾기에는 페이지의 a 태그가 너무 많다.

In [ ]:
current = html.find('a')
current

<a href="#lnb"><span>메뉴 영역으로 바로가기</span></a>

위와 같이 원하는 위치의 값이 아니다.

좀 더 구체적으로 찾기 위해서 class 이름을 알려주자

In [ ]:
current = html.find('a', class_="text _select_trigger _text")
current

<a aria-expanded="false" class="text _select_trigger _text" href="" onclick='return goOtherCR(this,"a=nco_x5e*2&amp;r=1&amp;i="+urlencode("0011AD9E_000001E7C267")+"&amp;u="+urlencode(this.href));'>1078회차 (2023.07.29.)</a>

class에 _를 붙여 위와 같이 검색하면 회차 정보가 담긴 원하는 위치의 tag를 검색이 가능하다.

find() 함수를 사용하여 찾은 결과도 html이기 때문에 tag 안에 tag를 찾는다면 find 함수를 연쇄적으로 사용하여 검색이 가능하다.

```
current = html.find(---).find(---)
```

찾은 current는 tag와 text로 이루어져 있다. 원하는 정보는 text 정보이므로 text만 추출해보자

In [ ]:
current.text

'1078회차 (2023.07.29.)'

text를 이용해 추출한 결과는 더이상 html이 아니라 문자열이다. 문자열 함수를 이용하여 회차 정보 중 숫자만 추출할 수 있다.

In [ ]:
current = int(current.text.split(' ')[0].replace('회차',''))
current

1078

- 공백을 기준으로 문자열을 나눈 후
- 만들어진 list에서 앞부분을 추출
- '회차' 문자를 replace 함수를 제거한다.
- 정수형으로 변환한다.

#### 로또 번호 찾기

In [ ]:
numbers = html.find('div',class_="win_ball").text.split()
numbers

['6', '10', '11', '14', '36', '38', '43']

리스트 안에 문자열이 담겨있는 형태로 저장되어 있으므로 각 요소를 map() 함수를 이용하여 정수형으로 변환하자

In [ ]:
numbers = list(map(int, numbers))
numbers

[6, 10, 11, 14, 36, 38, 43]

map 함수를 통해 각 변수를 int 형으로 변환하고, 이 값들을 다시 list로 변환한다.

#### 모든 회차의 정보 저장하기

"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=1077%ED%9A%8C%EB%A1%9C%EB%98%90"

위 url은 '1077회로또'를 검색한 결과이다.

여기서 1077 부분이 회차 정보를 담당하므로 이를 format 함수를 사용하여, 다른 회차로 대치 하면 모든 회차의 로또 정보를 얻을 수 있다.

(모든 회차정보를 얻는데는 시간이 많이 소요되므로 100회까지만 데이터를 획득해보자)

In [ ]:
total = []

for n in range(1, 101): # 1회차 부터 100회차까지
# for n in range(1, current + 1): # 1회차 부터 현재 회차까지
  url = requests.get(f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={n}%ED%9A%8C%EB%A1%9C%EB%98%90") # format 함수를 적용한다.
  if url.status_code != 200:
    print('get error... (code = {})'.format(url.status_code))
    continue

  html = BeautifulSoup(url.text)

  numbers = html.find('div',class_="win_ball").text.split()
  numbers = list(map(int, numbers))

  total.append(numbers)

  print('{}회 로또 데이터 저장중... '.format(n), numbers)

  time.sleep(0.3) # 트래픽 과부하 방지

1회 로또 데이터 저장중...  [10, 23, 29, 33, 37, 40, 16]
2회 로또 데이터 저장중...  [9, 13, 21, 25, 32, 42, 2]
3회 로또 데이터 저장중...  [11, 16, 19, 21, 27, 31, 30]
4회 로또 데이터 저장중...  [14, 27, 30, 31, 40, 42, 2]
5회 로또 데이터 저장중...  [16, 24, 29, 40, 41, 42, 3]
6회 로또 데이터 저장중...  [14, 15, 26, 27, 40, 42, 34]
7회 로또 데이터 저장중...  [2, 9, 16, 25, 26, 40, 42]
8회 로또 데이터 저장중...  [8, 19, 25, 34, 37, 39, 9]
9회 로또 데이터 저장중...  [2, 4, 16, 17, 36, 39, 14]
10회 로또 데이터 저장중...  [9, 25, 30, 33, 41, 44, 6]
11회 로또 데이터 저장중...  [1, 7, 36, 37, 41, 42, 14]
12회 로또 데이터 저장중...  [2, 11, 21, 25, 39, 45, 44]
13회 로또 데이터 저장중...  [22, 23, 25, 37, 38, 42, 26]
14회 로또 데이터 저장중...  [2, 6, 12, 31, 33, 40, 15]
15회 로또 데이터 저장중...  [3, 4, 16, 30, 31, 37, 13]
16회 로또 데이터 저장중...  [6, 7, 24, 37, 38, 40, 33]
17회 로또 데이터 저장중...  [3, 4, 9, 17, 32, 37, 1]
18회 로또 데이터 저장중...  [3, 12, 13, 19, 32, 35, 29]
19회 로또 데이터 저장중...  [6, 30, 38, 39, 40, 43, 26]
20회 로또 데이터 저장중...  [10, 14, 18, 20, 23, 30, 41]
21회 로또 데이터 저장중...  [6, 12, 17, 18, 31, 32, 21]
22회 로또 데이터 저장중...  [4, 5, 6, 

위 코드를 실행 중에 에러가 발생한다면 아래처럼 span tag를 모두 찾아 추가해야 한다.

In [ ]:
n = 1
url = requests.get(f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={n}%ED%9A%8C%EB%A1%9C%EB%98%90")
if url.status_code != 200:
    print('get error... (code = {})'.format(url.status_code))
html = BeautifulSoup(url.text)

numbers = html.find('div', class_="win_ball").find_all('span') # 모든 span tag를 찾는다
box = []

for i in numbers:
  box.append(int(i.text))

box

[10, 23, 29, 33, 37, 40, 16]

#### 데이터프레임화

In [ ]:
df = pd.DataFrame(total, columns = ['ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6','bonus'])
df

,ball1,ball2,ball3,ball4,ball5,ball6,bonus
0,10,23,29,33,37,40,16
1,9,13,21,25,32,42,2
2,11,16,19,21,27,31,30
3,14,27,30,31,40,42,2
4,16,24,29,40,41,42,3
...,...,...,...,...,...,...,...
95,1,3,8,21,22,31,20
96,6,7,14,15,20,36,3
97,6,9,16,23,24,32,43
98,1,3,10,27,29,37,11


In [ ]:
df.to_excel('lotto.xlsx')

### 증권 정보 예제

'네이버 증권' 코스피, 코스닥 데이터 크롤링

#### 사이트 분석

class=type_2인 table에 증권 정보가 나열되어 있음

In [8]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1')
html = BeautifulSoup(url.text)

table = html.find('table',class_='type_2')

table = pd.read_html(str(table)) # 문자열로 다시 변환하여 pandas를 이용하여 가져옴
table

[       N       종목명       현재가      전일비     등락률     액면가       시가총액      상장주식수  \
 0    NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
 1    1.0      삼성전자   67600.0    900.0  -1.31%   100.0  4035573.0  5969783.0   
 2    2.0  LG에너지솔루션  525000.0  11000.0  +2.14%   500.0  1228500.0   234000.0   
 3    3.0    SK하이닉스  118600.0   3300.0  -2.71%  5000.0   863411.0   728002.0   
 4    4.0  삼성바이오로직스  790000.0   3000.0  -0.38%  2500.0   562275.0    71174.0   
 ..   ...       ...       ...      ...     ...     ...        ...        ...   
 76  49.0    SK바이오팜   91600.0    100.0  +0.11%   500.0    71735.0    78313.0   
 77  50.0   삼성엔지니어링   36250.0    600.0  +1.68%  5000.0    71050.0   196000.0   
 78   NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
 79   NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
 80   NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
 
     외국인비율         거래량     PER    ROE 

pandas를 이용하여 표를 한번에 해석할 수 있다.

단 위와 같이 표를 list 형식으로 제공하므로 list의 0번 index에 접근하여 표만 추출할 수 있다

In [9]:
table = table[0]
table

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,67600.0,900.0,-1.31%,100.0,4035573.0,5969783.0,53.02,14631290.0,10.20,17.07,NaN
2,2.0,LG에너지솔루션,525000.0,11000.0,+2.14%,500.0,1228500.0,234000.0,5.08,327500.0,117.98,5.75,NaN
3,3.0,SK하이닉스,118600.0,3300.0,-2.71%,5000.0,863411.0,728002.0,52.55,2723598.0,-37.06,3.56,NaN
4,4.0,삼성바이오로직스,790000.0,3000.0,-0.38%,2500.0,562275.0,71174.0,10.59,40545.0,70.71,11.42,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49.0,SK바이오팜,91600.0,100.0,+0.11%,500.0,71735.0,78313.0,7.28,195614.0,-67.30,-36.66,NaN
77,50.0,삼성엔지니어링,36250.0,600.0,+1.68%,5000.0,71050.0,196000.0,50.73,617915.0,9.76,28.32,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN 값을 처리해보자

함부로 deopNa 함수를 사용하면 안된다. 행에 하나라도 NaN 값이 존재한다면 행 전체를 삭제해버리기 때문


In [12]:
del table['N'] # drop 함수를 사용해도 된다
del table['토론실']

table

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,삼성전자,67600.0,900.0,-1.31%,100.0,4035573.0,5969783.0,53.02,14631290.0,10.20,17.07
2,LG에너지솔루션,525000.0,11000.0,+2.14%,500.0,1228500.0,234000.0,5.08,327500.0,117.98,5.75
3,SK하이닉스,118600.0,3300.0,-2.71%,5000.0,863411.0,728002.0,52.55,2723598.0,-37.06,3.56
4,삼성바이오로직스,790000.0,3000.0,-0.38%,2500.0,562275.0,71174.0,10.59,40545.0,70.71,11.42
...,...,...,...,...,...,...,...,...,...,...,...
76,SK바이오팜,91600.0,100.0,+0.11%,500.0,71735.0,78313.0,7.28,195614.0,-67.30,-36.66
77,삼성엔지니어링,36250.0,600.0,+1.68%,5000.0,71050.0,196000.0,50.73,617915.0,9.76,28.32
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


회사 이름에 NaN 이 있다면 행을 삭제하는 식으로 NaN 값을 의도한대로 처리할 수 있다. 종목명은 무조건 공개되지만 다른 지표는 공개하지 않을 수도 있기 때문

In [13]:
table = table[table['종목명'].notnull()]
table

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
1,삼성전자,67600.0,900.0,-1.31%,100.0,4035573.0,5969783.0,53.02,14631290.0,10.20,17.07
2,LG에너지솔루션,525000.0,11000.0,+2.14%,500.0,1228500.0,234000.0,5.08,327500.0,117.98,5.75
3,SK하이닉스,118600.0,3300.0,-2.71%,5000.0,863411.0,728002.0,52.55,2723598.0,-37.06,3.56
4,삼성바이오로직스,790000.0,3000.0,-0.38%,2500.0,562275.0,71174.0,10.59,40545.0,70.71,11.42
5,POSCO홀딩스,597000.0,36000.0,+6.42%,5000.0,504890.0,84571.0,29.62,1856519.0,23.67,6.11
9,삼성전자우,55700.0,1000.0,-1.76%,100.0,458348.0,822887.0,72.88,756869.0,8.41,NaN
10,LG화학,615000.0,0.0,0.00%,5000.0,434143.0,70592.0,45.92,230182.0,29.51,6.95
11,삼성SDI,614000.0,3000.0,-0.49%,5000.0,422214.0,68765.0,48.92,204268.0,21.24,12.52
12,현대차,188400.0,400.0,+0.21%,5000.0,398525.0,211532.0,32.86,555929.0,5.73,9.36
13,NAVER,223000.0,11500.0,-4.90%,100.0,365829.0,164049.0,47.10,1919966.0,56.80,3.29
